In [ ]:
import numpy as np
import random as rnd

In [130]:
class SalesMan:
    def __init__(self, c, a):
        self.cities = c
        self.start = rnd.randint(0, c - 1)
        self.ants = a
        self.k = 10 * c
        self.d = self.init_cities(2 * 5)
        self.p = np.zeros((c, c))
        self.c = np.zeros((self.k, c))
        self.q = 1
        self.gamma = .1
        
        self.curiosity = 0.1
        self.alpha = 0.1
        self.beta = 0.1
    
    def init_cities(self, maxd):
        d = np.random.randint(-maxd, maxd, (self.cities, self.cities))
        d = np.abs((d + d.T) // 2) + 1
        np.fill_diagonal(d, 0)
        return d
        
        
    def run(self, it):
        for i in range(it):
            roads = [Ant(self.d, self.p, self.start).walk() for a in range(self.ants)]
            self.update_pheromones(roads)
    
    def update_pheromones(self, roads):
        self.p *= .9
        for road in roads:
            d = sum([c[1] for c in road])
            s = self.start
            
            for c in road[1:]:
                self.p[s][c[0]] += self.q / d
                self.p[c[0]][s] += self.q / d
                s = c[0]
            print(d, self.p)
    
class Ant:
    def __init__(self, c, p, s):
        self.cities = c
        self.p_map = p
        self.to_visit = [v for v in range(c.shape[0]) if not v == s]
        
        self.road = [(s, 0)]
    
    # def choice(self, ):
    
    def walk(self):
        while len(self.to_visit) > 0:
            crt = self.road[-1][0]
            ds = [self.cities[crt][v] for v in self.to_visit]
            
            i = rnd.randint(0, len(self.to_visit) - 1)
            c = self.to_visit.pop(i)
            self.road.append((c, self.cities[crt][c]))
        # print(*self.road)
        return self.road

In [131]:
salesman = SalesMan(5, 2)
salesman.run(2)

14 [[0.         0.         0.07142857 0.         0.        ]
 [0.         0.         0.         0.07142857 0.07142857]
 [0.07142857 0.         0.         0.         0.07142857]
 [0.         0.07142857 0.         0.         0.        ]
 [0.         0.07142857 0.07142857 0.         0.        ]]
17 [[0.         0.05882353 0.07142857 0.         0.05882353]
 [0.05882353 0.         0.05882353 0.07142857 0.07142857]
 [0.07142857 0.05882353 0.         0.         0.07142857]
 [0.         0.07142857 0.         0.         0.05882353]
 [0.05882353 0.07142857 0.07142857 0.05882353 0.        ]]
17 [[0.         0.11176471 0.06428571 0.05882353 0.05294118]
 [0.11176471 0.         0.05294118 0.06428571 0.12310924]
 [0.06428571 0.05294118 0.         0.         0.12310924]
 [0.05882353 0.06428571 0.         0.         0.05294118]
 [0.05294118 0.12310924 0.12310924 0.05294118 0.        ]]
12 [[0.         0.19509804 0.14761905 0.05882353 0.05294118]
 [0.19509804 0.         0.05294118 0.06428571 0.20644258]